# 🎮 Steam Store Scraper (0_plan with safe fallbacks)

Scrape 50 items with all possible columns → demo.csv
All missing fields default to None (NaN in Pandas).


In [ ]:
!pip install requests beautifulsoup4 pandas tqdm

In [1]:
import requests, time
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
url = "https://store.steampowered.com/search/?filter=topsellers&count=50"

r = requests.get(url, headers=headers)
soup = BeautifulSoup(r.text, 'html.parser')

rows = soup.select(".search_result_row")
games = []

for row in tqdm(rows[:50]):
    try:
        title = row.select_one(".title").get_text(strip=True) if row.select_one(".title") else None
        link = row.get("href") if row else None
        price = row.select_one(".discount_final_price")
        price = price.get_text(strip=True) if price else None
        release = row.select_one(".search_released")
        release = release.get_text(strip=True) if release else None
        status = row.select_one(".search_discount span")
        status = status.get_text(strip=True) if status else None
        image = row.select_one("img")['src'] if row.select_one("img") else None
        platforms = [p['class'][1] for p in row.select(".search_name .platform_img")] if row.select(".search_name .platform_img") else []

        # Detail page
        publisher = developer = description = review_summary = deal_name = deal_expiry = tags = None
        if link:
            detail_r = requests.get(link, headers=headers)
            detail_soup = BeautifulSoup(detail_r.text, 'html.parser')

            dev_pub = detail_soup.select(".dev_row")
            for row2 in dev_pub:
                header_elem = row2.select_one(".subtitle")
                header = header_elem.get_text(strip=True) if header_elem else ""
                if "Developer" in header:
                    developer = ", ".join([a.get_text(strip=True) for a in row2.select("a")]) or None
                if "Publisher" in header:
                    publisher = ", ".join([a.get_text(strip=True) for a in row2.select("a")]) or None

            desc = detail_soup.select_one(".game_description_snippet")
            description = desc.get_text(strip=True) if desc else None

            tag_elems = detail_soup.select(".glance_tags.popular_tags a")
            tags = ", ".join([t.get_text(strip=True) for t in tag_elems]) if tag_elems else None

            review = detail_soup.select_one(".user_reviews_summary_row")
            review_summary = review.get("data-tooltip-html", None).replace("<br>", " ") if review else None

            deal = detail_soup.select_one(".game_purchase_discount_quantity")
            deal_name = deal.get_text(strip=True) if deal else None
            expiry = detail_soup.select_one(".game_purchase_discount_countdown")
            deal_expiry = expiry.get_text(strip=True) if expiry else None

        games.append({
            "title": title,
            "status": status,
            "category": None,  # not always on listing
            "tags": tags,
            "release_date": release,
            "price": price,
            "is_free": (price and "Free" in price),
            "platforms": ", ".join(platforms) if platforms else None,
            "publisher": publisher,
            "developer": developer,
            "description": description,
            "review_summary": review_summary,
            "deal_name": deal_name,
            "deal_expiry": deal_expiry,
            "image_url": image,
            "store_url": link
        })
        time.sleep(1)
    except Exception as e:
        print("Error scraping row:", e)

df = pd.DataFrame(games)
df.to_csv("demo.csv", index=False)
df.head()

100%|██████████| 50/50 [01:29<00:00,  1.80s/it]


,title,status,category,tags,release_date,price,is_free,platforms,publisher,developer,description,review_summary,deal_name,deal_expiry,image_url,store_url
0,PUBG: BATTLEGROUNDS,None,None,"Survival, Shooter, Battle Royale, Multiplayer,...","21 Dec, 2017",Free,True,None,"KRAFTON, Inc.",PUBG Corporation,"PUBG: BATTLEGROUNDS, the high-stakes winner-ta...","69% of the 14,331 user reviews in the last 30 ...",None,None,https://shared.akamai.steamstatic.com/store_it...,https://store.steampowered.com/app/578080/PUBG...
1,Europa Universalis V,None,None,"Resource Management, 4X, Grand Strategy, Tradi...","4 Nov, 2025","₹3,259.00",False,None,Paradox Interactive,Paradox Tinto,"Use war, trade or diplomacy to satisfy your gr...","77% of the 1,695 user reviews in your language...",None,None,https://shared.akamai.steamstatic.com/store_it...,https://store.steampowered.com/app/3450310/Eur...
2,ARC Raiders,None,None,"Extraction Shooter, PvP, PvE, Third-Person Sho...","30 Oct, 2025","₹2,467.00",False,None,Embark Studios,Embark Studios,ARC Raiders is a multiplayer extraction advent...,"90% of the 30,400 user reviews in your languag...",None,None,https://shared.akamai.steamstatic.com/store_it...,https://store.steampowered.com/app/1808500/ARC...
3,Counter-Strike 2,None,None,"FPS, Shooter, Multiplayer, Competitive, Action...","21 Aug, 2012",Free,True,None,Valve,Valve,"For over two decades, Counter-Strike has offer...","81% of the 83,875 user reviews in the last 30 ...",None,None,https://shared.akamai.steamstatic.com/store_it...,https://store.steampowered.com/app/730/Counter...
4,Apex Legends™,None,None,"Free to Play, Multiplayer, Battle Royale, FPS,...","4 Nov, 2020",Free,True,None,Electronic Arts,Respawn,"Apex Legends is the award-winning, free-to-pla...","66% of the 4,652 user reviews in the last 30 d...",None,None,https://shared.akamai.steamstatic.com/store_it...,https://store.steampowered.com/app/1172470/Ape...


✅ Done: `demo.csv` created with 50 rows and all possible columns (missing values = NaN).


In [3]:
import requests
from bs4 import BeautifulSoup

headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"}
url = "https://store.steampowered.com/search/?filter=topsellers&count=3"

resp = requests.get(url, headers=headers)
soup = BeautifulSoup(resp.text, "html.parser")

rows = soup.select(".search_result_row")[:3]

for i, row in enumerate(rows, start=1):
    print(f"\n=== RAW GAME BLOCK {i} ===\n")
    print(row.prettify())   # dumps the full HTML for that game
    print("\n" + "="*60 + "\n")



=== RAW GAME BLOCK 1 ===

<a class="search_result_row ds_collapse_flag" data-ds-appid="578080" data-ds-crtrids="[33973721,44295939]" data-ds-descids="[2,5]" data-ds-itemkey="App_578080" data-ds-steam-deck-compat-handled="true" data-ds-tagids="[1662,1774,176981,3859,1663,1775,3814]" data-gpnav="item" data-search-page="1" href="https://store.steampowered.com/app/578080/PUBG_BATTLEGROUNDS/?snr=1_7_7_7000_150_1" onmouseout="HideGameHover( this, event, 'global_hover' )" onmouseover="GameHover( this, event, 'global_hover', {&quot;type&quot;:&quot;app&quot;,&quot;id&quot;:578080,&quot;public&quot;:1,&quot;v6&quot;:1} );">
 <div class="search_capsule">
  <img src="https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/578080/14f42ab797699a9c9f137fda66582d8776da4330/capsule_231x87.jpg?t=1758255634"/>
 </div>
 <div class="responsive_search_name_combined">
  <div class="search_name ellipsis">
   <span class="title">
    PUBG: BATTLEGROUNDS
   </span>
  </div>
  <div class="search_pla